In [2]:
## Imports

# utility modules
import glob
import os
import sys
import re


# the usual suspects:
import numpy as np
import matplotlib.pyplot as plt

# specialty modules
import h5py
import pyproj
import iris

# run matplotlib in 'widget' mode
%matplotlib widget
%load_ext autoreload
%autoreload 2

In [3]:
cd

/home/jovyan


In [4]:
cd Land_Ice_Applications/readers

/home/jovyan/Land_Ice_Applications/readers


In [5]:
from read_HDF5_ATL03 import read_HDF5_ATL03
from get_ATL03_x_atc import get_ATL03_x_atc

### 1.2.2 Data used
Data for this tutorial are stored in a shared drive, accessible to all tutorial participants.  If you're getting data for yourself, you'll need to put it in a consistent place, and change this cell to match your directory.

In [6]:
data_root='shared/FirnAndMelt/ATL03/Antarctica/'

In [7]:
import rasterio
import xarray as xr
import cartopy.crs as ccrs


In [8]:
cd

/home/jovyan


In [9]:
cd FirnAndMelt/

/home/jovyan/FirnAndMelt


I've  also added a Landsat 8 image from 2020-01-20 that overlaps the area of interest on the Amery Ice Shelf. I've used Moussavi et al (2020)'s automated lake detection algorithm to identify lake pixels. 

In [10]:
landsat8 = xr.open_rasterio('LC08_L1GT_127111_20200120_20200128_01_T2_All_Masks.tif')
landsat8

<xarray.DataArray (band: 1, y: 7381, x: 7291)>
[53814871 values with dtype=float64]
Coordinates:
  * band     (band) int64 1
  * y        (y) float64 8.025e+05 8.025e+05 8.024e+05 ... 5.811e+05 5.811e+05
  * x        (x) float64 1.712e+06 1.712e+06 1.712e+06 ... 1.93e+06 1.930e+06
Attributes:
    transform:         (30.0, 0.0, 1711785.0, 0.0, -30.0, 802515.0)
    crs:               +init=epsg:3031
    res:               (30.0, 30.0)
    is_tiled:          1
    nodatavals:        (nan,)
    scales:            (1.0,)
    offsets:           (0.0,)
    AREA_OR_POINT:     Point
    TIFFTAG_SOFTWARE:  MATLAB 9.8, Mapping Toolbox 4.10

In [11]:
x  =  landsat8.x.values.ravel()
y  =  landsat8.y.values.ravel()
xx,yy =  np.meshgrid(x,y)

In [12]:
SouthPolarStereo  = pyproj.CRS(landsat8.crs)
WGS84 = pyproj.CRS("EPSG:4326")


/srv/conda/envs/notebook/lib/python3.7/site-packages/pyproj/crs/crs.py:280: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(projparams)


In [13]:
from pyproj import Transformer
transformer = Transformer.from_crs(SouthPolarStereo, WGS84)
lat,long = transformer.transform(xx,yy)

In [31]:
lat_ds  = xr.DataArray(lat, name='lat', coords=(landsat8.y,landsat8.x))
long_ds  = xr.DataArray(long, name='long', coords=(landsat8.y,landsat8.x))



In [ ]:
Landsat8 =  xr.concat([landsat8,lat_ds,long_ds], dim='x')

In [35]:
Landsat8

<xarray.DataArray (newdim: 3, band: 1, y: 7381, x: 7291)>
array([[[[  0.        ,   0.        ,   0.        , ...,   0.        ,
            0.        ,   0.        ],
         [  0.        ,   0.        ,   0.        , ...,   0.        ,
            0.        ,   0.        ],
         [  0.        ,   0.        ,   0.        , ...,   0.        ,
            0.        ,   0.        ],
         ...,
         [  0.        ,   0.        ,   0.        , ...,   0.        ,
            0.        ,   0.        ],
         [  0.        ,   0.        ,   0.        , ...,   0.        ,
            0.        ,   0.        ],
         [  0.        ,   0.        ,   0.        , ...,   0.        ,
            0.        ,   0.        ]]],


       [[[-72.72655601, -72.72631143, -72.72606685, ..., -70.92974844,
          -70.92950021, -70.92925198],
         [-72.72667067, -72.72642609, -72.7261815 , ..., -70.92985163,
          -70.9296034 , -70.92935517],
         [-72.72678533, -72.72654074, -72.72629615, ..., -70.92995482,
          -70.92970658, -70.92945835],
         ...,
         [-73.47289257, -73.4726363 , -73.47238004, ..., -71.5986449 ,
          -71.59838701, -71.59812912],
         [-73.47297957, -73.4727233 , -73.47246704, ..., -71.59872254,
          -71.59846465, -71.59820675],
         [-73.47306656, -73.4728103 , -73.47255403, ..., -71.59880017,
          -71.59854228, -71.59828438]]],


       [[[ 64.88259253,  64.88297845,  64.88336436, ...,  67.42693674,
           67.42725234,  67.42756794],
         [ 64.88341575,  64.88380166,  64.88418755, ...,  67.42769595,
           67.42801155,  67.42832713],
         [ 64.88423897,  64.88462487,  64.88501076, ...,  67.42845517,
           67.42877075,  67.42908633],
         ...,
         [ 71.247533  ,  71.24783867,  71.24814433, ...,  73.2455503 ,
           73.24579608,  73.24604185],
         [ 71.24843336,  71.24873902,  71.24904466, ...,  73.24636671,
           73.24661248,  73.24685824],
         [ 71.24933373,  71.24963937,  71.24994501, ...,  73.24718314,
           73.24742889,  73.24767464]]]])
Coordinates:
  * y        (y) float64 8.025e+05 8.025e+05 8.024e+05 ... 5.811e+05 5.811e+05
  * band     (band) int64 1
  * x        (x) float64 1.712e+06 1.712e+06 1.712e+06 ... 1.93e+06 1.930e+06
Dimensions without coordinates: newdim
Attributes:
    transform:         (30.0, 0.0, 1711785.0, 0.0, -30.0, 802515.0)
    crs:               +init=epsg:3031
    res:               (30.0, 30.0)
    is_tiled:          1
    nodatavals:        (nan,)
    scales:            (1.0,)
    offsets:           (0.0,)
    AREA_OR_POINT:     Point
    TIFFTAG_SOFTWARE:  MATLAB 9.8, Mapping Toolbox 4.10

In [56]:
%matplotlib widget
L8lakes = landsat8.where(landsat8.values==1)
L8lakes.plot(x=L8lakes.long,y=L8lakes.lat)


AttributeError: 'DataArray' object has no attribute 'long'

# 2. Data in along-track format

# 2.1 ATL03 elevations

Before we start looking at the ATL06 data we've downloaded, let's have a look at some of the ATL03 data that were used to make them.  One of the source ATL03 files is in the shared folder, and we'll read it with Tyler Sutterley's excellent "read_HDF5_ATL03" function.

In [11]:
cd 

/home/jovyan


In [12]:
cd 'shared/FirnAndMelt/ATL03/Antarctica/'

/srv/shared/FirnAndMelt/ATL03/Antarctica


In [13]:
# read the data:
#rgt="0027"
#cycle="06"
# read the IS2 data with Tyler's ATL03 reader:
ATL03_file=  'processed_ATL03_20200203123745_05920612_003_01.h5'
IS2_atl03_mds, IS2_atl03_attrs, IS2_atl03_beams =read_HDF5_ATL03(ATL03_file)

The data are returned in a set of dictionaries that mimic the structure of an ATL03 file.  To help visualize the data, we're going to calculate an along-track coordinate for every photon in the cloud (x_atc).  This is a slightly complex job, and there's a helper function in the readers directory that you can look at if you want the details.

In [14]:
# add x_atc to the ATL03 data structure (this function adds to the LS2_ATL03_mds dictionary)
get_ATL03_x_atc(IS2_atl03_mds, IS2_atl03_attrs, IS2_atl03_beams)

### 2.1.1 Plotting ATL03 photons
Now let's plot the ATL03 photons.  We'll plot all the photon heights as small black dots, then plot the photons that the ATL03 land-ice signal finder designates as surface (with low, medium, or high confidence) in green.  

In [15]:
#-- select the 2l beam from ATL03
D3 = IS2_atl03_mds['gt2r']

#-- create scatter plot of photon data (e.g., photon elevation vs x_atc)
fig=plt.figure(figsize=(6,4))
ax = fig.add_subplot(111)
ax.plot(D3['heights']['lat_ph'], D3['heights']['h_ph'],'k.',markersize=0.25, label='all photons')
LMH=D3['heights']['signal_conf_ph'][:,3] >= 2
ax.plot(D3['heights']['lat_ph'][LMH], D3['heights']['h_ph'][LMH],'g.',markersize=0.5, label='flagged photons')
h_leg=ax.legend()

ax.set_xlabel('x_atc, m')
ax.set_ylabel('h, m')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

What we see here is a fat black bar, representing all the photons that were close to the surface (in a ~200 m window), with a green line representing the photons flagged by ATL03 as representing the surface. If we zoom in (using the square button on the right) we can see that the black bar is made up of lots of individual photons, with a darker line that represents the surface.  

In [ ]:
plt.figure()
ax = plt.axes(projection=ccrs.SouthPolarStereo())
L8lakes.plot(ax  = ax, transform = ccrs.SouthPolarStereo(), cmap='Greys_r')
ax.set_extent([67.5, 68.5, -72.3, -72], ccrs.PlateCarree())
plt.scatter(D3['heights']['lon_ph'], D3['heights']['lat_ph'], transform=ccrs.PlateCarree())

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### 2.1.2 Cloudy tracks and singnal-finding blunders

In [16]:

TEP=(D3['heights']['signal_conf_ph'][:,3] <-1)   
BG=(D3['heights']['signal_conf_ph'][:,3] ==0)   |  (D3['heights']['signal_conf_ph'][:,3] ==1)
LMH=D3['heights']['signal_conf_ph'][:,3] >= 2


In [17]:
%matplotlib widget
plt.plot(D3['heights']['x_atc'][LMH], D3['heights']['h_ph'][LMH],'g.',markersize=0.5, label='flagged photons')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
import pandas as pd

In [20]:
conda install pyresample

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.2
  latest version: 4.8.3

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /srv/conda/envs/notebook

  added / updated specs:
    - pyresample


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.4.5.2 |       hecda079_0         147 KB  conda-forge
    certifi-2020.4.5.2         |   py37hc8dfbb8_0         152 KB  conda-forge
    configobj-5.0.6            |             py_0          31 KB  conda-forge
    pykdtree-1.3.1             |py37h03ebfcd_1003          64 KB  conda-forge
    pyresample-1.16.0          |   py37h0da4684_0         594 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         988 KB

The following NEW package

In [21]:
import pyresample as prs

In [22]:
selection = pd.Series(data = D3['heights']['h_ph'][LMH], index = D3['heights']['x_atc'][LMH])

In [51]:
is2_geom  =  prs.geometry.SwathDefinition(lons=D3['heights']['lon_ph'], lats=D3['heights']['lat_ph'])
lake_geom  = prs.geometry.SwathDefinition(lons  = L8lakes['x'], lats  =  L8lakes['y'], )

TypeError: __init__() got an unexpected keyword argument 'x'

In [64]:
selection_var  = selection.rolling(100).var()

In [65]:
%matplotlib widget
selection_var.plot()
selection_var.where(selection_var.values>1).plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in greater
  This is separate from the ipykernel package so we can avoid doing imports until


In [66]:
lake_idx  = np.argwhere(selection_var.values>1)

/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in greater
  """Entry point for launching an IPython kernel.


In [67]:
%matplotlib widget
plt.plot(D3['heights']['x_atc'][LMH], D3['heights']['h_ph'][LMH],'g.',markersize=0.5, label='flagged photons')
plt.plot(D3['heights']['x_atc'][LMH][lake_idx], D3['heights']['h_ph'][LMH][lake_idx],'b.',markersize=0.5, label='flagged photons')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …